 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [85]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [86]:
# Imports to get us started
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [87]:
# an initial look at the data
print(df.shape, '\n')
print(df.isnull().sum(), '\n')
df.head()

(748, 5) 

months_since_last_donation     0
number_of_donations            0
total_volume_donated           0
months_since_first_donation    0
made_donation_in_march_2007    0
dtype: int64 



,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [88]:
# our target value is whether or not a participant made a donation in March (1, 0)
y = df['made_donation_in_march_2007']
majcl = y.mode()[0]
y_pred = np.full(shape=df.shape[0], fill_value=majcl)

# The majority class with a binary classification is simply the 
# percentage of the mode in relation to the total observations
print('Majority Class Baseline: ', accuracy_score(y, y_pred), '\n')
print('Majority Class v.s. Minority as % of total :')

df['made_donation_in_march_2007'].value_counts(normalize=True)

Majority Class Baseline:  0.7620320855614974 

Majority Class v.s. Minority as % of total :


0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [89]:
print(classification_report(y, y_pred));

             precision    recall  f1-score   support

          0       0.76      1.00      0.86       570
          1       0.00      0.00      0.00       178

avg / total       0.58      0.76      0.66       748



/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The recall for each is just the True Positives (TP) divided by the sum of the True Positives and False Negatives. It could also be called the true positive rate. 

## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [90]:
X = df.drop(columns='made_donation_in_march_2007')
y = df['made_donation_in_march_2007']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [91]:
pipe = make_pipeline(StandardScaler(), SelectKBest(f_classif), LogisticRegression())

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [92]:
param_grid = {
    'selectkbest__k' : [ 1, 2, 3, 4],
    'logisticregression__class_weight' : [None, 'balanced'],
    'logisticregression__C' : [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

gs = GridSearchCV(pipe, 
                  param_grid= param_grid, 
                  cv= 5, 
                  scoring= 'recall', 
                  verbose=1)

gs.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:    2.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x1a20437f28>)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'selectkbest__k': [1, 2, 3, 4], 'logisticregression__class_weight': [None, 'balanced'], 'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=1)

## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [93]:
print('Best cross-validation score : ', gs.best_score_, '\n')
print('Best parameters : ')
# gs.best_params_

for i in gs.best_params_:
    print(i, ':',  gs.best_params_[i])

Best cross-validation score :  0.8152337858220211 

Best parameters : 
logisticregression__C : 0.0001
logisticregression__class_weight : balanced
selectkbest__k : 2


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

In [94]:
# accuracy = (TruePositives+TrueNegatives)/AllOutcomes
accuracy = (36+85)/(85+58+8+36)
accuracy

0.6470588235294118

Calculate precision

In [95]:
# precision = TruePositives/(TruePositives+FalsePositives)
precision = 36/(36+58)
precision

0.3829787234042553

Calculate recall

In [96]:
# recall = TruePositives/(TruePositives+FalseNegatives) --> Sensitivity or True Positive Rate
recall = 36/(36+8)
recall

0.8181818181818182

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 

### Part 1 --> Polynomial Features

In [97]:
poly = PolynomialFeatures(degree=2)
X_train_polynomial = poly.fit_transform(X_train)

print(X_train.shape, X_train_polynomial.shape)

In [98]:
pipe = make_pipeline(StandardScaler(), 
                     SelectKBest(f_classif),
                     LogisticRegression())

In [103]:
param_grid = {
    'selectkbest__k' : [ 1, 2, 3, 4],
    'logisticregression__class_weight' : [None, 'balanced'],
    'logisticregression__C' : [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

gs = GridSearchCV(pipe, 
                  param_grid= param_grid, 
                  cv= 5, 
                  scoring= 'recall', 
                  verbose=1)

gs.fit(X_train_polynomial, y_train)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

...

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/

/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/Users/lambda_school_loaner_32/anaconda3/lib/python3.7/site-packages/


Cross-Validation Score: -0.8152337858220211

Best estimator: Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=4, score_func=<function f_classif at 0x1a20437f28>)), ('logisticregression', LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])



### Part 3:


In [100]:
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
months_since_last_donation
number_of_donations

Features not selected:
total_volume_donated
months_since_first_donation


In [102]:
test_score = gs.score(X_test, y_test)
print('Test Score:', test_score)

Test Score: 0.8333333333333334


### Part 4:
#### F1 Score and False Positive Rate

In [77]:
# Harmonic Mean of Precision and Recall
F1 = 2*(precision*recall)/(precision+recall)
f'F1 Score: {F1}'

'F1 Score: 0.5217391304347826'

In [78]:
# False Positives divided by False Positives and True Negatives
FPR = (58)/(85+58)
f'False Positive Rate: {FPR}'

'False Positive Rate: 0.40559440559440557'